## Bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

import pandas as pd
import numpy as np

from datetime import datetime
import string
from lxml import etree

import time

## Inicializar navegador

In [2]:
# Opciones para el webdriver
options = webdriver.ChromeOptions()
# options.add_argument("--headless")

# Iniciar sesión del navegador
driver = webdriver.Chrome(options = options)

## Utilidades

In [3]:
def xpath(xpath):
    elements = driver.find_elements(By.XPATH, xpath)
    if len(elements) > 0:
        if len(elements) == 1:
            return(elements[0])
        else:
            return(elements)
    else:
        return("Elemento no encontrado")

In [4]:
def dataframe_to_dict(dataframe):
    """
    Permite convertir un dataframe a un
    diccionario de Python
    """
    # Convertir a dataframe
    data_dict = dataframe.to_dict("records")
    return(data_dict)

## Subrutinas

In [5]:
def login_sigmaa(id_user, password):
    """
    Permite iniciar sesion en el SIGMAA 
    usando las credenciales del estudiante
    
    Regresa `true` si el incio de sesión se pudo concretar
    Regresa `false` si no se pudo iniciar sesión
    """
    
    # Acceder al SIGMAA
    url = 'https://uclb.ucaribe.edu.mx/sigmaav2/'
    driver.get(url)

    # Buscar el campo de usuario y escribir el username
    id_user_input = xpath("/html/body/div[2]/form/div/span[2]/input")
    id_user_input.clear()
    id_user_input.send_keys(id_user)
    
    # Buscar el campo de contraseña y escribir el password
    password_input = xpath("/html/body/div[2]/form/div/input")
    password_input.clear()
    password_input.send_keys(password)
    
    # Buscar el botón de submit y dar clic para iniciar sesión
    submit_input = driver.find_element(By.XPATH, "/html/body/div[2]/form/button")
    submit_input.click()

    # Si se intenta loguear pero se queda en la misma página de logueo
    if driver.title == "Ingresar · SIGMAA - Unicaribe":
        # Inicio de sesion erroneo
        return(False)
    else:
        # Inicio de sesion exitoso
        return(True)

In [6]:
def logout_sigmaa():
    """
    Permite cerrar la sesión de un usuario en SIGMAA
    """
    # Encontrar el botón de cierre de sesión
    logout = xpath("/html/body/div[2]/div/form/a")
    # Clic para cerrar sesión
    logout.click()

### Información personal

In [7]:
def informacion_personal(id_user, password):
    """
    Permite recuperar la información personal del estudiante
    en esa sección en SIGMAA
    """
    # Dirigirse a la sección de información personal
    personal_information_tab = xpath("/html/body/div[1]/div/div/div/div/div/ul[2]/li[6]/a")
    personal_information_tab.click()
    
    # Recuperar los datos de la tabla con la información
    # del estudiante
    df = pd.read_html(driver.page_source)
    # print(len(df))
    df = df[1]
    
    # Determinar si existen registros de beca
    scholarship = False if xpath("/html/body/center/table/tbody/tr[1]/th").text == "Datos personales" else True
    
    # Recuperar la información dependiendo de la estructura
    # de la página en donde se encuentran los datos
    if scholarship == True:
        username = xpath("/html/body/center/table/tbody/tr[6]/td[3]").text
        first_lastname = xpath("/html/body/center/table/tbody/tr[6]/td[1]").text
        second_lastname = xpath("/html/body/center/table/tbody/tr[6]/td[2]").text
        curp = xpath("/html/body/center/table/tbody/tr[7]/td[2]").text
        rfc = xpath("/html/body/center/table/tbody/tr[7]/td[1]").text
        nationality = xpath("/html/body/center/table/tbody/tr[8]/td[2]").text
        nss = xpath("/html/body/center/table/tbody/tr[9]/td[1]").text
        personal_email = xpath("/html/body/center/table/tbody/tr[9]/td[2]").text
        birthday = xpath("/html/body/center/table/tbody/tr[8]/td[3]").text
        sex = xpath("/html/body/center/table/tbody/tr[7]/td[3]").text
        personal_phone = xpath("/html/body/center/table/tbody/tr[11]/td[2]").text
        home_phone = xpath("/html/body/center/table/tbody/tr[11]/td[1]").text
        marital_status = Select(xpath("/html/body/center/table/tbody/tr[8]/td[1]/select")).first_selected_option.text
        personal_address = xpath("/html/body/center/table/tbody/tr[10]/td[1]").text
        birthplace_country = Select(xpath("/html/body/center/table/tbody/tr[16]/td/select")).first_selected_option.text
        birthplace_state = Select(xpath("//*[@id='uiNacEstado']/select")).first_selected_option.text
        birthplace_city = Select(xpath("//*[@id='uiNacCiudad']/select")).first_selected_option.text
        first_relative = xpath("/html/body/center/table/tbody/tr[14]/td[1]").text
        second_relative = xpath("/html/body/center/table/tbody/tr[14]/td[2]").text
        relatives_marital_status = Select(xpath("/html/body/center/table/tbody/tr[14]/th[4]/select")).first_selected_option.text
        highschool_country = Select(xpath("/html/body/center/table/tbody/tr[20]/td/select")).first_selected_option.text
        highschool_state = Select(xpath("//*[@id='uiEscEstado']/select")).first_selected_option.text
        highschool_region = xpath("/html/body/center/table/tbody/tr[22]/td[1]").text
        highschool_city = xpath("/html/body/center/table/tbody/tr[22]/td[2]").text
        highschool_name = xpath("/html/body/center/table/tbody/tr[23]/td").text
        highschool_campus = xpath("/html/body/center/table/tbody/tr[24]/td[1]").text
        school_system = xpath("/html/body/center/table/tbody/tr[24]/td[2]").text
        working_status = xpath("/html/body/center/table/tbody/tr[12]/td[1]").text
        company_name = xpath("/html/body/center/table/tbody/tr[12]/td[2]").text
        company_address = xpath("/html/body/center/table/tbody/tr[12]/td[3]").text
        company_phone = xpath("/html/body/center/table/tbody/tr[11]/td[3]").text
    
    else:
        username = xpath("/html/body/center/table/tbody/tr[2]/td[3]").text
        first_lastname = xpath("/html/body/center/table/tbody/tr[2]/td[1]").text
        second_lastname = xpath("/html/body/center/table/tbody/tr[2]/td[2]").text
        curp = xpath("/html/body/center/table/tbody/tr[3]/td[2]").text
        rfc = xpath("/html/body/center/table/tbody/tr[3]/td[1]").text
        nationality = xpath("/html/body/center/table/tbody/tr[4]/td[2]").text
        nss = xpath("/html/body/center/table/tbody/tr[5]/td[1]").text
        personal_email = xpath("/html/body/center/table/tbody/tr[5]/td[2]").text
        birthday = xpath("/html/body/center/table/tbody/tr[4]/td[3]").text
        sex = xpath("/html/body/center/table/tbody/tr[3]/td[3]").text
        personal_phone = xpath("/html/body/center/table/tbody/tr[7]/td[2]").text
        home_phone = xpath("/html/body/center/table/tbody/tr[7]/td[1]").text
        marital_status = Select(xpath("/html/body/center/table/tbody/tr[4]/td[1]/select")).first_selected_option.text
        personal_address = xpath("/html/body/center/table/tbody/tr[6]/td[1]").text
        birthplace_country = Select(xpath("/html/body/center/table/tbody/tr[12]/td/select")).first_selected_option.text
        birthplace_state = Select(xpath("//*[@id='uiNacEstado']/select")).first_selected_option.text
        birthplace_city = Select(xpath("//*[@id='uiNacCiudad']/select")).first_selected_option.text
        first_relative = xpath("/html/body/center/table/tbody/tr[10]/td[1]").text
        second_relative = xpath("/html/body/center/table/tbody/tr[10]/td[2]").text
        relatives_marital_status = Select(xpath("/html/body/center/table/tbody/tr[10]/th[4]/select")).first_selected_option.text
        highschool_country = Select(xpath("/html/body/center/table/tbody/tr[16]/td/select")).first_selected_option.text
        highschool_state = Select(xpath("//*[@id='uiEscEstado']/select")).first_selected_option.text
        highschool_region = xpath("/html/body/center/table/tbody/tr[18]/td[1]").text
        highschool_city = xpath("/html/body/center/table/tbody/tr[18]/td[2]").text
        highschool_name = xpath("/html/body/center/table/tbody/tr[19]/td").text
        highschool_campus = xpath("/html/body/center/table/tbody/tr[20]/td[1]").text
        school_system = xpath("/html/body/center/table/tbody/tr[20]/td[2]").text
        working_status = xpath("/html/body/center/table/tbody/tr[8]/td[1]").text
        company_name = xpath("/html/body/center/table/tbody/tr[8]/td[2]").text
        company_address = xpath("/html/body/center/table/tbody/tr[8]/td[3]").text
        company_phone = xpath("/html/body/center/table/tbody/tr[7]/td[3]").text
        
    # Dirigirse a la seccion de la boleta escolar
    grades_tab = xpath("/html/body/div[1]/div/div/div/div/div/ul[2]/li[4]/a")
    grades_tab.click()
    
    # Recuperar la información de la carrera
    career_name = xpath("/html/body/center/table[1]/tbody/tr[2]/td").text.split(" / ")[1]
    situation = xpath("/html/body/center/table[1]/tbody/tr[3]/td").text
    status = xpath("/html/body/center/table[1]/tbody/tr[4]/td").text
    social_service = xpath("/html/body/center/table[1]/tbody/tr[6]/td").text
    study_plan = xpath("/html/body/center/table[1]/tbody/tr[2]/td").text.split(" / ")[0]
    
    # Obtener el departamento de acuerdo a la carrera
    department = "Ciencias Básicas e Ingenierías"
    
    personal_information = {
        # Información personal básica
        "id_user" : id_user,
        "password" : password,
        "username" : string.capwords(username),
        "first_lastname" : string.capwords(first_lastname),
        "second_lastname" : string.capwords(second_lastname),
        "profile_picture" : "default_avatar.jpg",
        "curp" : curp,
        "rfc" : rfc,
        "nationality" : nationality,
        "nss" : nss,
        "personal_email" : personal_email.lower(),
        "birthday" : birthday,
        "sex" : sex,
        "personal_phone" : personal_phone,
        "home_phone" : home_phone,
        "marital_status" : marital_status,
        "personal_address" : personal_address.title(),
        
        # Información de nacimiento
        "birthplace" : {
            "country" : birthplace_country.capitalize(),
            "state" : birthplace_state,
            "city" : birthplace_city
        },
        
        # Información de familiares
        "relatives" : {
            "first_relative" : first_relative.title(),
            "second_relative" : second_relative.title(),
            "relatives_marital_status" : relatives_marital_status
        },
        
        # Información de la escuela de procedencia
        "highschool" : {
            "country" : highschool_country.capitalize(),
            "state" : highschool_state,
            "region" : highschool_region,
            "city" : highschool_city,
            "school_name" : highschool_name,
            "campus" : highschool_campus,
            "school_system" : school_system 
        },
        
        # Información de la carrera
        "career" : {
            "career_name" : career_name,
            "situation" : situation,
            "status" : status,
            "social_service" : social_service,
            "study_plan" : study_plan,
            "department" : department
        },
        
        # Información laboral (IMPORTANTE)
        "job" : {
            "status" : working_status,
            "company" : company_name,
            "address" : company_address,
            "phone" : company_phone
        },
        
        # Última fecha de actualización
        "last_updated" : datetime.utcnow()
    }
    
    return(personal_information)

### Boletas de calificaciones

In [8]:
def promedios_calificaciones(calificaciones):
    """
    Permite obtener el promedio de las calificaciones
    para las asignaturas cuantificables del periodo
    """

    # Lista de calificaciones finales cuantificables
    # en el periodo
    quantifiables = []

    # Para cada calificacion final
    for grade in calificaciones["final_grade"]:
        # Si es una asignatura cuantificable (numérica)
        if grade not in ["Aprobado", "Reprobado", "-"]:
            # Añadir a la lista de cuantificables
            quantifiables.append(float(grade))
    
    # Promedio de calificaciones por default
    mean = "-"

    # Si hay registros de calificaciones cuantificables
    if len(quantifiables) > 0:
        # Calcular el promedio y redondear al segundo decimal
        mean = round(np.mean(quantifiables), 2)

    return(mean)

In [9]:
def calificaciones(id_user):
    """
    Permite obtener las boletas las boletas de calificaciones
    disponibles para el estudiante en la sección de horario/boleta
    """

    # Dirigirse a la seccipon de la boleta escolar
    school_grades_tab = xpath("/html/body/div[1]/div/div/div/div/div/ul[2]/li[4]/a")
    school_grades_tab.click()

    # Recuperar el dropdown para cambiar los periodos de consulta
    periods_dropdown = Select(xpath("/html/body/center/form/select"))

    # Obtener todos los periodos del menú desplegable
    periods = [[period.get_attribute("value"), period.text] for period in periods_dropdown.options][1:]

    # Diccionario que contendrá las calificaciones por periodos
    periods_grades = {}

    # Iterar sobre los periodos disponibles en menú desplegable
    for period_code, period_name in periods:
        # Recuperar el dropdown cada vez que se consulta
        # ya que el web element cambia de dirección cuando se refresca
        # la página
        periods_dropdown = Select(xpath("/html/body/center/form/select"))
        
        # Seleccionar el periodo actual
        periods_dropdown.select_by_value(period_code)
        
        # Consultar el periodo actual
        button = xpath("/html/body/center/form/input[4]")
        button.click()
        
        # Recuperar las calificaciones en la consulta actual
        df = pd.read_html(driver.page_source)[1]
        
        # Nombre para las columnas
        column_names = ["number", "type", "section", "subject", "first_partial", "second_partial", "third_partial", "mean", "final_grade", "U1", "U2"]
        
        # Limpiar el dataframe de las calificaciones actuales
        # Cambiar el nombre de las columnas
        df = df.set_axis(column_names, axis = "columns")
        
        # Eliminar columnas no deseadas
        del df["U1"]
        del df["U2"]
        
        # Eliminar filas con valores nulos
        df.dropna(how = "all", inplace = True)
        
        # Recuperar el html de la vista actual
        html = etree.HTML(driver.page_source)
        
        # Obtenemos la lista de claves de asignatura
        id_subjects = html.xpath("//table[2]/tbody/tr/td[contains(@align, 'left')]/text()[1]")

        # Obtenemos los nombres de las asignaturas
        subjects = html.xpath("//table[2]/tbody/tr/td[contains(@align, 'left')]/b/text()[1]")

        # Obtenemos los profesores de cada asignatura
        tr_nodes = html.xpath("//table[2]/tbody/tr/td[contains(@align, 'left')]/text()[3]")
        teachers = [node.split("\n")[2].split("            ")[1] for node in tr_nodes]

        # Obtenemos las modalidades
        modalities = html.xpath("//table[2]/tbody/tr/td[contains(@align, 'left')]/span[contains(@style, 'color:#08c;')]/text()")
        
        # Sobreescribir los nombres de asignaturas
        df["subject"] = subjects
        # Insertar nuevas columnas para las claves, docentes y modalidades
        df.insert(3, "id_subject", id_subjects)
        df.insert(4, "teacher", teachers)
        df.insert(5, "modality", modalities)
        
        # Imputar valores faltantes con un guión
        df = df.fillna("-")

        # Insertar las calificaciones en una lista
        periods_grades[period_code] = {
            # Calcular el promedio del periodo actual
            "mean" : promedios_calificaciones(df),
            # Insertar los registros de calificaciones del periodo actual
            "grades" : dataframe_to_dict(df)
        }

    # Consolidar la información en un diccionario con información 
    # del usuario y las calificaciones para cada periodo encontrado
    student_grades = {
        "id_user" : id_user,
        "periods" : periods_grades,
        "last_updated" : datetime.utcnow()
    }

    return(student_grades)

### Oferta académica

In [10]:
def oferta_academica(study_plan, career):
    # Dirigirse a la sección de la oferta académica
    academic_offer_tab = xpath("/html/body/div[1]/div/div/div/div/div/ul[2]/li[1]/a")
    academic_offer_tab.click()

    # xpath de pestañas
    tabs = [
        # Secciones
        "/html/body/center/ul/li[1]/a",
        # Talleres
        "/html/body/center/ul/li[2]/a",
        # Lengua Extranjera
        "/html/body/center/ul/li[3]/a"
    ]

    title = xpath("/html/body/center/div").text

    academic_offer = {
        "study_plan" : study_plan,
        "career" : career,
        "title" : title
    }

    # Recorrer los tabs para sección de oferta
    for index, tab in enumerate(tabs):

        section = ""
        if index == 0:
            section = "additionals"
        elif index == 1:
            section = "workshops"
        else:
            section = "foreign_languages"

        # Intercambiar de pestaña en cada iteración
        current_tab = xpath(tab)
        current_tab.click()

        # Recuperar las tablas con la información de 
        # la oferta en la sección actual. 
        # NOTA: Es una lista de dataframes, el indice 0 corresponde
        # al titulo de la oferta del periodo actual. Los dataframes con 
        # las asignaturas ofertadas se encuentran a partir del
        # indice 1 en adelante.
        df = pd.read_html(driver.page_source)[1:]

        # Variable para controlar el table data
        # Servirá más adelante para obtener la posición con 
        # el nombre del docente de cada asignatura
        td = 0

        # La estructura de las columnas cambia de acuerdo
        # al tipo de oferta académica. En el caso de las asignaturas
        # adicionales, existe una columna para el tipo
        if section == "additionals":
            # La columna de las asignaturas está en la columna 4 para adicionales
            td = 4
            # Nombres para las columnas
            column_names = ['type', 'id_subject', 'section', 'subject', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'U1', 'U2', 'U3', 'U4']
            # Columnas que se desean conservar
            desired_columns = ['type', 'id_subject', 'section', 'subject', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday']

        else:
            # La columna de las asignaturas está en la columna 3 para ingles y talleres
            td = 3
            # Nombres para las columnas
            column_names = ['id_subject', 'section', 'subject', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'U1', 'U2', 'U3', 'U4', 'U5']
            # Columnas que se desean conservar
            desired_columns = ['id_subject', 'section', 'subject', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday']
            
        # Recorrer cada dataframe en la pestaña actual
        for i in range(len(df)):
            # Renombrar las columnas de la tabla de acuerdo a la sección
            df[i] = df[i].set_axis(column_names, axis = "columns")

            # Filtrar las columnas de interés
            df[i] = df[i].iloc[:, df[i].columns.isin(desired_columns)]

            # Guardar el contenido de cada tabla para
            # cada indice en el diccionario que contiene la oferta
            # académica
            # Si es la primera tabla en la lista
            if i == 0:
                # Insertar las asignaturas por primera vez
                academic_offer[section] = df[i]
            else:
                # Concatenar con los datos existentes
                academic_offer[section] = pd.concat([academic_offer[section], df[i]], ignore_index = True)

        # Recuperar el html de la pestaña actual
        html = etree.HTML(driver.page_source)

        # Se necesita separar el docente y el nombre de asignatura
        # de la columna subject

        # Buscamos las asignaturas, docentes y modalidades usando XPATH
        subjects = html.xpath("//table[contains(@class, 'datos')]/tbody/tr/td[" + str(td) + "]/b/text()")
        # Lista de asignaturas en todas las tablas de la sección actual
        subjects = [subject.split("\n")[0] for subject in subjects]

        tr_nodes = html.xpath("//table[contains(@class, 'datos')]/tbody/tr/td[" + str(td) + "]/text()[2]")
        # Lista de docentes en todas las tablas de la sección actual
        teachers = [tr.split("\n                        ")[0] for tr in tr_nodes]

        # Lista de modalidades de asignatura en la sección actual
        modalities = html.xpath("//table[contains(@class, 'datos')]/tbody/tr/td[" + str(td) + "]/span[contains(@style, 'color:#08c;')]/text()")
            
        # Reasignar la columna con las asignaturas separadas de los demas
        academic_offer[section]["subject"] = subjects
        # Insertar una nueva columna con los docentes
        academic_offer[section].insert(3, "teacher", teachers)
        # Insertar una nueva columna con las modalidades de asignaturas
        academic_offer[section].insert(4, "modality", modalities)
        
        # Si la sección no son adicionales
        if section != "additionals":
            # Se inserta el tipo ya que solo aparece en adicionales pero no en
            # los demas tipos de asignaturas en la oferta
            academic_offer[section].insert(0, "type", "")

        # Convertir el dataframe de la sección actual a diccionario
        academic_offer[section] = dataframe_to_dict(academic_offer[section])
        
    # Última fecha de actualización
    academic_offer["last_updated"] = datetime.utcnow()
    return(academic_offer)

### Historial de pagos

## Testing

In [11]:
id_user = "190300453"
password = "Isaacperez16"

# Iniciar sesión en SIGMAA
login_sigmaa(id_user, password)

True

In [12]:
user = informacion_personal(id_user, password)
user

{'id_user': '190300453',
 'password': 'Isaacperez16',
 'username': 'Isaac',
 'first_lastname': 'Perez',
 'second_lastname': 'Amayo',
 'profile_picture': 'default_avatar.jpg',
 'curp': 'PEAI011007HQRRMSA8',
 'rfc': '',
 'nationality': 'Mexicano',
 'nss': '17190151898',
 'personal_email': 'isaacperezamayo@gmail.com',
 'birthday': '2001-10-07',
 'sex': 'Masculino',
 'personal_phone': '9983219777',
 'home_phone': '9',
 'marital_status': 'Soltero(a)',
 'personal_address': 'Sm259, M 4, Lt 1, Calle El Limonero 4',
 'birthplace': {'country': 'Mexico',
  'state': 'Quintana Roo',
  'city': 'Chetumal'},
 'relatives': {'first_relative': 'Omel Perez Zapata',
  'second_relative': 'Esther Amayo Romero',
  'relatives_marital_status': 'Casado(a)'},
 'highschool': {'country': 'Mexico',
  'state': 'Quintana Roo',
  'region': 'Benito Juárez',
  'city': 'Cancún',
  'school_name': 'Centro de Bachillerato Tecnológico Industrial y de Servicios',
  'campus': 'Núm. 111',
  'school_system': 'Público'},
 'career'

In [13]:
academic_offer = oferta_academica(user["career"]["study_plan"], user["career"]["career_name"])
academic_offer

{'study_plan': '2016ID',
 'career': 'Ingeniería en Datos e Inteligencia Organizacional',
 'title': 'Oferta Académica Primavera 2023',
 'additionals': [{'type': 'BAS',
   'id_subject': 'ID0102',
   'section': 1,
   'teacher': 'Kauil Castillo / Gener Antonio',
   'modality': 'Asignatura Presencial',
   'subject': 'Física clásica',
   'monday': '-',
   'tuesday': 'G-15 15:00-17:00',
   'wednesday': '-',
   'thursday': '-',
   'friday': 'G-11 15:00-17:00',
   'saturday': '-'},
  {'type': 'BAS',
   'id_subject': 'II0209',
   'section': 1,
   'teacher': 'Canul Chalé / Juan Diego',
   'modality': 'Asignatura Presencial',
   'subject': 'Cálculo integral',
   'monday': 'G-16 17:00-19:00',
   'tuesday': '-',
   'wednesday': 'G-15 16:00-17:00',
   'thursday': '-',
   'friday': '-',
   'saturday': '-'},
  {'type': 'BAS',
   'id_subject': 'IT0104',
   'section': 1,
   'teacher': 'Naredo García / Enrique',
   'modality': 'Asignatura Presencial',
   'subject': 'Matemáticas discretas',
   'monday': '-

In [14]:
grades = calificaciones(user["id_user"])
grades

{'id_user': '190300453',
 'periods': {'202204': {'mean': '-',
   'grades': [{'number': 1.0,
     'type': 'OP1',
     'section': 1.0,
     'id_subject': 'PID0302',
     'teacher': 'García Fernández / Alejandro',
     'modality': 'Asignatura Presencial',
     'subject': 'Prácticas profesionales II',
     'first_partial': '-',
     'second_partial': '-',
     'third_partial': '-',
     'mean': '-',
     'final_grade': '-'}]},
  '202203': {'mean': 9.75,
   'grades': [{'number': 1.0,
     'type': 'OP1',
     'section': 1.0,
     'id_subject': 'IT0210',
     'teacher': 'Morales Saavedra / Emmanuel',
     'modality': 'Asignatura Presencial',
     'subject': 'Programación orientada a objetos',
     'first_partial': 10.0,
     'second_partial': 8.0,
     'third_partial': 10.0,
     'mean': 9.33333302,
     'final_grade': 9.0},
    {'number': 2.0,
     'type': 'OP1',
     'section': 1.0,
     'id_subject': 'ID3418',
     'teacher': 'Treviño Arzápalo / Ángel Alejandro',
     'modality': 'Asignatu

In [15]:
# Cerrar sesión en SIGMAA
logout_sigmaa()

In [16]:
# Cerrar sesión de navegador
driver.quit()

# Modelos

In [30]:
from pydantic import BaseModel

In [ ]:
# Modelos para las calificaciones
class SubjectGrades(BaseModel):
    number : int
    type : str
    section : int
    id_subject : str
    teacher : str
    modality : str
    subject : str
    first_partial : int | str
    second_partial : int | str
    third_partial : int | str
    mean : float | str
    final_grade : float | str

class Period(BaseModel):
    mean : float | str
    grades : list[SubjectGrades]

class Periods(BaseModel):
    period : dict(Period)

class Grades(BaseModel):
    id_user : str
    periods : Periods

## Mapas curriculares

In [19]:
study_plans = ["2016ID", "2021ID", "2019IA"]

curricular_map = "2016ID"
data = pd.read_excel("../../resources/Mapas curriculares.xlsx", sheet_name = curricular_map)

In [22]:
data[["school_year", "credits"]].groupby(["school_year"]).sum(["credits"]).reset_index()

,school_year,credits
0,1,102
1,2,116
2,3,70
3,4,54
4,"3, 4",104
5,Cultural,0
6,Deportiva,0
7,Innovación en TIC,40
8,Inteligencia Organizacional y de Negocios,40
9,Lengua extranjera,0


In [23]:
validation = pd.read_excel("../../resources/Mapas curriculares.xlsx", sheet_name=curricular_map+"_credits")

In [26]:
validation.groupby(["school_year"]).sum("credits").reset_index()

,school_year,max_credits,req_credits
0,1,102,90
1,2,116,104
2,3,70,70
3,4,54,54
4,"3, 4",104,48
5,Cultural,0,0
6,Deportiva,0,0
7,Innovación en TIC,40,40
8,Inteligencia Organizacional y de Negocios,40,40
9,Lengua extranjera,0,0
